In [31]:
import pandas as pd
import numpy as np
from dplython import (DplyFrame, X, diamonds, select, sift, sample_n,
                      sample_frac, head, arrange, mutate, group_by, summarize, DelayFunction,
                      ungroup) 

from dplython.dplython import gather, spread

In [2]:
combine = DplyFrame(pd.read_csv('/Users/kiefer/github-repos/DSI-SF-2/datasets/football_combine/combine.csv'))

In [4]:
combine.head()

,id,Year,Name,Position,HeightFeet,HeightInches,Weight,Arms,Hands,FortyYD,...,Bench,Round,College,Pick,PickRound,PickTotal,FirstName,LastName,HeightInchesTotal,Wonderlic
0,8984,2013,Quanterus Smith,DE,6,5.0,250,33.25,10.375,0.0,...,0,5,Western Kentucky,13(146),13,146,Quanterus,Smith,77.0,0
1,9002,2013,Abry Jones,DT,6,3.0,313,35.00,9.750,0.0,...,30,0,NaN,NaN,0,0,Abry,Jones,75.0,0
2,9004,2013,Bennie Logan,DT,6,2.0,309,34.00,10.250,0.0,...,30,3,LSU,5(67),5,67,Bennie,Logan,74.0,0
3,9012,2013,John Boyett,FS,5,10.0,204,30.50,8.500,0.0,...,27,6,Oregon,24(192),24,192,John,Boyett,70.0,0
4,9018,2013,Bacarri Rambo,FS,6,0.0,211,31.00,9.250,0.0,...,17,6,Georgia,23(191),23,191,Bacarri,Rambo,72.0,0


In [44]:
combine = combine >> sift(X.Name != 'Ben Obomanu') >> select(X.Name, X.HeightFeet, X.HeightInches, X.Weight)


In [45]:
tmp = DplyFrame(tmp)
type(tmp)
tmp = combine >> gather('variable', 'value', [X.HeightFeet, X.HeightInches])
type(tmp)

pandas.core.frame.DataFrame

In [46]:
tmp = DplyFrame(tmp)
tmp = tmp >> spread(X.variable, X.value)

In [47]:
tmp.head()

,Name,Weight,HeightFeet,HeightInches
0,A.C. Leonard,252,6.0,2.0
1,A.J. Edds,246,6.0,4.0
2,A.J. Green,211,6.0,4.0
3,A.J. Greene,298,6.0,5.0
4,A.J. Hawk,248,6.0,1.0


In [23]:
df_columns = df.columns.values.tolist()
df_columns

['Name', 'Weight', 'variable', 'value']

In [25]:
spread_index_columns = [col for col in df_columns if col not in [df.variable._name, df.value._name]]
spread_index_columns

['Name', 'Weight']

In [27]:
def new_spread_cols(df, cols):
    temp_index_for_reshape = ['' for i in range(len(df))]
    for col in cols:
        temp_index_for_reshape += df[col].map(str)
    return temp_index_for_reshape

In [29]:
temp_columns = new_spread_cols(df, spread_index_columns)
temp_columns[0:5]

0    Quanterus Smith250
1         Abry Jones313
2       Bennie Logan309
3        John Boyett204
4      Bacarri Rambo211
Name: Name, dtype: object

In [35]:
out_df = (df >> ungroup()).assign(temp_index_for_reshape=temp_columns)
out_df.head()

,Name,Weight,variable,value,temp_index_for_reshape
0,Quanterus Smith,250,HeightFeet,6.0,Quanterus Smith250
1,Abry Jones,313,HeightFeet,6.0,Abry Jones313
2,Bennie Logan,309,HeightFeet,6.0,Bennie Logan309
3,John Boyett,204,HeightFeet,5.0,John Boyett204
4,Bacarri Rambo,211,HeightFeet,6.0,Bacarri Rambo211


In [36]:
out_df = out_df.set_index('temp_index_for_reshape')
out_df.head()

,Name,Weight,variable,value
temp_index_for_reshape,,,,
Quanterus Smith250,Quanterus Smith,250,HeightFeet,6.0
Abry Jones313,Abry Jones,313,HeightFeet,6.0
Bennie Logan309,Bennie Logan,309,HeightFeet,6.0
John Boyett204,John Boyett,204,HeightFeet,5.0
Bacarri Rambo211,Bacarri Rambo,211,HeightFeet,6.0


In [37]:
new_spread_data = out_df[[tmp.variable._name, tmp.value._name]]
new_spread_data.head()

,variable,value
temp_index_for_reshape,,
Quanterus Smith250,HeightFeet,6.0
Abry Jones313,HeightFeet,6.0
Bennie Logan309,HeightFeet,6.0
John Boyett204,HeightFeet,5.0
Bacarri Rambo211,HeightFeet,6.0


In [39]:
tmp2 = new_spread_data.groupby([new_spread_data.index, tmp.variable._name]).agg('count').reset_index()

In [41]:
tmp2[tmp2.value > 1]

,temp_index_for_reshape,variable,value
746,Ben Obomanu205,HeightFeet,2
747,Ben Obomanu205,HeightInches,2


In [38]:
if not all(new_spread_data.groupby([new_spread_data.index, tmp.variable._name]).agg('count').reset_index().value < 2):
    raise ValueError('Duplicate identifers')

ValueError: Duplicate identifers

In [ ]:
  
new_data = new_spread_data.pivot(columns=key._name, values=values._name)
old_data = out_df[spread_index_columns].drop_duplicates()
output_data = old_data.merge(new_data, left_index=True, right_index=True).reset_index(drop=True)
return output_data

In [7]:
# Select specific columns of the DataFrame using select, and 
# get the first few using head
tmp = combine >> select(X.Position, X.Weight, X.Hands)
print tmp.shape
tmp.head()

(4625, 3)


,Position,Weight,Hands
0,DE,250,10.375
1,DT,313,9.750
2,DT,309,10.250
3,FS,204,8.500
4,FS,211,9.250


In [8]:
# Filter out rows using sift
tmp = combine >> sift(X.Weight < 200) >> select(X.Position, X.Weight, X.Hands, X.PickTotal)
print tmp.shape
tmp.head()

(797, 4)


,Position,Weight,Hands,PickTotal
15,CB,182,9.375,123
49,RB,193,0.000,0
55,WR,192,9.000,54
56,CB,183,9.000,0
58,CB,190,0.000,156


In [9]:
# Sample with sample_n or sample_frac,
tmp = (combine >> 
       sample_n(10) >>
       select(X.Position, X.Weight, X.Hands, X.PickTotal))

print tmp.shape
tmp

(10, 4)


,Position,Weight,Hands,PickTotal
1787,FB,230,0.000,191
3368,TE,251,0.000,184
493,FS,206,9.750,75
3149,OLB,235,0.000,0
4519,OT,291,0.000,0
1736,TE,242,8.625,0
2257,OLB,234,0.000,227
3496,DT,297,0.000,64
3028,QB,219,0.000,0
1860,RB,212,0.000,0


In [10]:
# sort with arrange
tmp = (combine >> 
       sample_n(10) >>
       arrange(X.Weight) >> 
       select(X.Position, X.Weight, X.Hands, X.PickTotal))
tmp

,Position,Weight,Hands,PickTotal
1888,CB,192,0.000,0
1615,WR,193,9.625,84
1340,RB,211,0.000,0
1638,RB,225,0.000,219
763,ILB,231,0.000,130
1956,QB,237,10.375,0
2879,OLB,240,0.000,207
2151,ILB,241,9.375,120
3078,TE,262,0.000,81
12,OT,315,10.250,0


In [11]:
# You can: 
# add columns with mutate (referencing other columns!)
# group rows into dplyr-style groups with group_by
# collapse rows into single rows using sumarize
tmp = (combine >> 
       sample_n(10) >>
       mutate(mean_of_weight_hands=(X.Weight+X.Hands)/2.) >> 
       select(X.Position, X.Weight, X.Hands, X.PickTotal, X.mean_of_weight_hands))
tmp


,Position,Weight,Hands,PickTotal,mean_of_weight_hands
212,FS,213,0.00,6,106.500
271,CB,193,9.00,27,101.000
514,FS,214,0.00,157,107.000
3185,DT,293,0.00,12,146.500
224,WR,172,8.75,100,90.375
1048,CB,198,0.00,29,99.000
2988,QB,222,0.00,0,111.000
660,WR,179,0.00,118,89.500
608,CB,191,0.00,0,95.500
2104,RB,202,0.00,144,101.000


In [12]:
# group by columns
tmp = (combine >>
       group_by(X.Position, X.Year) >> 
       summarize(avg_weight=X.Weight.mean()) >>
       arrange(X.Year, X.Position) >>
       select(X.Position))
tmp.head(10)




,Position
1,CB
17,DE
33,DT
49,FB
63,FS
79,ILB
98,OC
113,OG
129,OLB
145,OT


In [13]:
# To pass the DataFrame or columns into functions, apply @DelayFunction
@DelayFunction
def MeanOfColumns(df):
    meancol = df.mean(axis=1)
    return meancol

tmp = (combine >>
       select(X.Year, X.Weight, X.Hands) >>
       mutate(Mean_of_cols = (combine >> 
                              select(X.Year, X.Weight, X.Hands) >> 
                              MeanOfColumns(X))))
tmp

,Year,Weight,Hands,Mean_of_cols
0,2013,250,10.375,757.791667
1,2013,313,9.750,778.583333
2,2013,309,10.250,777.416667
3,2013,204,8.500,741.833333
4,2013,211,9.250,744.416667
5,2013,243,10.500,755.500000
6,2013,302,10.250,775.083333
7,2013,332,9.875,784.958333
8,2013,320,9.750,780.916667
9,2013,243,9.375,755.125000
